In [1]:
pip install ultralytics opencv-python pillow

Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import os
from ultralytics import YOLO
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
from collections import Counter

# 🧠 Define carnivores
carnivores = ['cat', 'dog', 'lion', 'tiger', 'wolf', 'fox']

# 🧠 Load YOLOv8 model
model = YOLO("yolov8n.pt")  # You can use your custom-trained model path here

# ⚙️ Initialize GUI Window
root = tk.Tk()
root.title("Animal Detection System")
root.geometry("500x300")
root.configure(bg="#1e1e1e")

# 🔍 Function to detect animals in an image
def detect_image(image_path):
    img = cv2.imread(image_path)
    results = model(img)[0]
    detected_carnivores = []

    for box in results.boxes:
        cls_id = int(box.cls[0])
        label = model.names[cls_id]
        x1, y1, x2, y2 = map(int, box.xyxy[0])

        if label.lower() in carnivores:
            color = (0, 0, 255)  # Red for carnivores
            detected_carnivores.append(label.lower())
        else:
            color = (255, 0, 0)  # Blue for others

        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    # Show carnivore count popup
    if detected_carnivores:
        count = dict(Counter(detected_carnivores))
        popup_text = "\n".join([f"{k.title()}: {v}" for k, v in count.items()])
        messagebox.showinfo("⚠️ Carnivores Detected!", popup_text)

    # Show detection output
    cv2.imshow("Animal Detection", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# 🎥 Function to detect animals in a video
def detect_video(video_path):
    cap = cv2.VideoCapture(video_path)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame)[0]
        detected_carnivores = []

        for box in results.boxes:
            cls_id = int(box.cls[0])
            label = model.names[cls_id]
            x1, y1, x2, y2 = map(int, box.xyxy[0])

            if label.lower() in carnivores:
                color = (0, 0, 255)
                detected_carnivores.append(label.lower())
            else:
                color = (255, 0, 0)

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        cv2.imshow("Animal Detection - Video", frame)

        # Press 'q' to quit video early
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    # Popup with total carnivores
    if detected_carnivores:
        count = dict(Counter(detected_carnivores))
        popup_text = "\n".join([f"{k.title()}: {v}" for k, v in count.items()])
        messagebox.showinfo("⚠️ Carnivores Detected in Video!", popup_text)

    cap.release()
    cv2.destroyAllWindows()

# 📂 Browse and detect file
def browse_file():
    file_path = filedialog.askopenfilename(filetypes=[("Media Files", "*.jpg *.png *.mp4")])
    if not file_path:
        return

    ext = os.path.splitext(file_path)[1].lower()
    if ext in [".jpg", ".png"]:
        detect_image(file_path)
    elif ext == ".mp4":
        detect_video(file_path)
    else:
        messagebox.showerror("❌ Invalid File", "Please select a .jpg, .png or .mp4 file.")

# 🖱️ GUI Button
btn = tk.Button(root, text="📁 Upload Image or Video", font=("Arial", 16), command=browse_file, bg="#0f62fe", fg="white", padx=10, pady=5)
btn.pack(pady=100)

# 🟢 Start GUI Loop
root.mainloop()



0: 640x640 1 cat, 100.0ms
Speed: 4.2ms preprocess, 100.0ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)
